<a href="https://colab.research.google.com/github/macgyver121/DADS6005/blob/main/Quiz2/Pyspark_join_multiple_dstreams_json2dataframe_mmbk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://mirrors.estointernet.in/apache/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark

In [2]:
%%capture
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json

Ex1

In [5]:
def readMyStream(rdd):
  if not rdd.isEmpty():
    df = spark.read.json(rdd)
    print(df.dtypes)
    print('Started the Process')
    print('Selection of Columns')
    df = df.select('firstname','lastname')
    df.show()

Ex2

In [6]:
def readMyStream2(rdd):
  if not rdd.isEmpty():
    df = spark.read.json(rdd)
    df.show()

    # The inferred schema can be visualized using the printSchema() method
    #df.printSchema()

    # Creates a temporary view using the DataFrame
    #df.createOrReplaceTempView("people")

    # SQL statements can be run by using the sql methods provided by spark
    #resultDF = spark.sql("SELECT * FROM people")
    #resultDF.show()

In [7]:
from pyspark.sql import functions as func

In [33]:
def readMyStream3(rdd1,rdd2):
    if not rdd1.isEmpty():
        df1 = spark.read.json(rdd1)
        df2 = spark.read.json(rdd2)
        #df1.show()
        #df2.show()
        
        df3 = df1.join(df2,['id'])
        #df3.show()
        df3.select('id', 'country', 'num_click').show()
        df3.groupBy('country').sum('num_click').show()
        #df3.createOrReplaceTempView("table")
        #resultDF = spark.sql("SELECT sum(num_click) FROM table")
        #resultDF.show()

In [34]:
kafka_topic_name = "my-first-topic-mmbk8"
kafka_topic_name2 = "my-first-topic2-mmbk8"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [35]:
sc = pyspark.SparkContext()
spark = SparkSession(sc)
ssc = StreamingContext(sc,2) # Show results every 2 seconds

In [36]:
kvs1 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'
                        })

kvs2 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name2], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'
                        })

#kvs1.pprint()
#kvs2.pprint()
parsed = kvs1.map(lambda x: x[1])
#parsed.foreachRDD( lambda rdd: readMyStream2(rdd) )

#parsed.pprint()
parsed2 = kvs2.map(lambda x: x[1])
#parsed2.pprint()

parsed.transformWith(readMyStream3,parsed2).window(8,4).pprint()
#parsed3.pprint()

#parsed.fullOuterJoin(parsed2).pprint(30)


#pairs.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 30, 10)

ssc.start()
ssc.awaitTerminationOrTimeout(60)
ssc.stop()
sc.stop()

+---+-------+---------+
| id|country|num_click|
+---+-------+---------+
|  5|  Japan|        7|
|  3|England|       10|
|  4|England|        6|
+---+-------+---------+

+-------+--------------+
|country|sum(num_click)|
+-------+--------------+
|  Japan|             7|
|England|            16|
+-------+--------------+

+---+--------+---------+
| id| country|num_click|
+---+--------+---------+
|  7|Thailand|        2|
|  6|   Japan|        3|
|  8|   Japan|        5|
+---+--------+---------+

+--------+--------------+
| country|sum(num_click)|
+--------+--------------+
|Thailand|             2|
|   Japan|             8|
+--------+--------------+

-------------------------------------------
Time: 2022-11-25 05:12:00
-------------------------------------------

+---+-------+---------+
| id|country|num_click|
+---+-------+---------+
|  5|  Japan|        7|
|  3|England|       10|
|  4|England|        6|
+---+-------+---------+

+-------+--------------+
|country|sum(num_click)|
+-------+----

In [32]:
ssc.stop()
sc.stop()

-------------------------------------------
Time: 2022-11-25 05:11:08
-------------------------------------------

